In [4]:
import re
import bs4
from bs4 import BeautifulSoup
from requests import get

import pandas as pd

In [5]:
def search_author(term):
    result = []
    term = term.replace(' ','+')
    
    url = f'https://sinta.ristekbrin.go.id/authors?q={term}&view=&search=1&ag='
    html = get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    
    for num, i in enumerate(soup.select(".text-blue")): #get first item
        #get name and sinta id
        name = i.text.strip()
        author_id = re.findall(r'\d+', i['href'])[0]
        # get affiliation and NIDN
        for x in i.find_next('dd'):
            affil_name = x.contents[0] 
            NIDN = x.find_next('dd').contents[2].replace(' : ', '')
        # Get expertise
        areas = [] # empty container
        expertise1 = i.find_next('a', {'class':"area-item-small"}) # get first expertise
        ctr = 0 # helper counter
        while ctr < 3 and isinstance(expertise1, bs4.element.Tag): # loop until NoneType & max 3 expertise
            areas.append(expertise1.contents[0])
            expertise1 = expertise1.find_next('a') # get next expertise
            if expertise1['class'][0] == 'area-item-small': # if expertise feature, continue
                ctr = ctr + 1
            else: # if not expertise feature pass
                ctr = 3     
        
        result_data = {'id': author_id,
                       'name': name,
                       'NIDN' : NIDN,
                       'affiliation': affil_name,
                       'areas': areas 
                       }
        result.append(result_data)
    return result

In [19]:
data = search_author('Agus Zainal Arifin')
data = pd.DataFrame(data)
data

,id,name,NIDN,affiliation,areas
0,5975467,AGUS ZAINAL ARIFIN,0009087205,Institut Teknologi Sepuluh Nopember,"[computer vision, image processing, informatio..."
1,6693261,KIAGUS ZAINAL ARIFIN,0013045907,Politeknik Negeri Sriwijaya,[Palembang]


In [22]:
import sinta_scraper as sinta
authors = sinta.authors(data.id)#[data.affiliation == 'Institut Teknologi Sepuluh Nopember'].id)
pd.DataFrame(authors)

,id,name,url,affiliation,department,areas,score,rank,scopus,scholar,wos,sinta,books,ipr
0,6693261,KIAGUS ZAINAL ARIFIN,http://sinta.ristekbrin.go.id/authors/detail?i...,"{'id': '542', 'name': 'Politeknik Negeri Sriwi...",,[Palembang],"{'overall': 0.06, '3_years': 0.0, 'overall_v2'...","{'national': 148059, '3_years_national': 15094...","{'documents': 0, 'citations': 0, 'h-index': 0,...","{'documents': 4, 'citations': 6, 'h-index': 2,...","{'documents': 0, 'citations': None, 'h-index':...","{'S0': 0, 'S1': 0, 'S2': 0, 'S3': 0, 'S4': 0, ...",0,0
1,5975467,AGUS ZAINAL ARIFIN,http://sinta.ristekbrin.go.id/authors/detail?i...,"{'id': '417', 'name': 'Institut Teknologi Sepu...",Teknik Informatika,"[computer vision, image processing, informatio...","{'overall': 42.08, '3_years': 1.1, 'overall_v2...","{'national': 823, '3_years_national': 1054, 'a...","{'documents': 58, 'citations': 413, 'h-index':...","{'documents': 284, 'citations': 1308, 'h-index...","{'documents': 1, 'citations': None, 'h-index':...","{'S0': 1, 'S1': 3, 'S2': 3, 'S3': 3, 'S4': 5, ...",0,2


In [25]:
authors[0]['url']

'http://sinta.ristekbrin.go.id/authors/detail?id=6693261&view=overview'